**About** : This notebook is used to train models.

In [ ]:
# %load_ext nb_black
%load_ext autoreload
%autoreload 2

In [ ]:
cd ../src/

## Initialization

### Imports

In [ ]:
import os
import torch

print(torch.__version__)
os.environ['CUDA_VISIBLE_DEVICES'] = "0"
device = torch.cuda.get_device_name(0)
print(device)

In [ ]:
import os
import sys
import glob
import json
import cudf
import torch
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from tqdm import tqdm
from sklearn.metrics import *

pd.set_option('display.width', 500)
pd.set_option('max_colwidth', 100)

In [ ]:
from training.main import k_fold

from utils.logger import (
    prepare_log_folder,
    save_config,
    create_logger,
    init_neptune
)

from params import *
from utils.plots import *
from data.dataset import *
from data.preparation import *
from utils.torch import init_distributed
from utils.plots import plot_sample
from model_zoo.models import define_model

## Data

In [ ]:
df = prepare_data(DATA_PATH, processed_folder="processed_3/")

In [ ]:
df = df.head(10000)

In [ ]:
dataset = SignDataset(df, max_len=None, train=True)
# dataset = SignDataset(df, max_len=None, train=False)

In [ ]:
data = dataset[0]

In [ ]:
plot_sample(data)

In [ ]:
lens = []
for i in tqdm(range(len(df.head(10000)))):
    lens.append(np.load(dataset.paths[i]).shape[0])
sns.histplot(lens)

In [ ]:
# df[['len', 'sign']].groupby('sign').mean()

In [ ]:
from training.losses import *

In [ ]:
loss = SignLoss({"name": "ce", "aux_loss_weight": 0})

In [ ]:
loss(torch.rand(64, 250), 0, torch.ones(64), 0)

## Model

In [ ]:
import seaborn as sns 
from utils.torch import count_parameters, load_model_weights
from numerize.numerize import numerize

In [ ]:
model = define_model("bert_deberta", transfo_heads=8, embed_dim=128, transfo_dim=128*3).cuda()

In [ ]:
# model.type_embed.weight.data.fill_(0.)

In [ ]:
# model = load_model_weights(model, "../logs/2023-03-10/12/bert_deberta_0.pt")

In [ ]:
numerize(count_parameters(model))

In [ ]:
dataset = SignDataset(df, max_len=30, train=False)

In [ ]:
BS = 32

x = {}
batch = [dataset[idx] for idx in np.random.randint(len(dataset), size=BS)]
for k in data:
    x[k] = torch.cat([d[k].unsqueeze(0).cuda() for d in batch])

In [ ]:
y, _ = model(x)
y.size(), y.max()

### Att

In [ ]:
atts, _ = model.get_landmark_attentions(x)

In [ ]:
order = np.argsort(x['type'][0][0].cpu().numpy())
att = torch.cat(atts, 1)
print(att.size())
att = att.max(0).values.sum(0).detach().cpu().numpy()

print(att.shape)
att = att[order].T[order].T

att -= att.min()
att = att.mean(0)

In [ ]:
x['type'].size()

In [ ]:
cols = np.array([list(c) + [1] for c in sns.color_palette("muted", n_colors=12)])

plt.figure(figsize=(15, 5))
plt.bar(x=range(len(att)), height=att, color=cols[np.sort(x['type'][0][0].cpu().numpy()) - 1])
plt.xlim(-1, len(att))

for i, (col, m) in enumerate(zip(cols, TYPE_MAPPING.keys())):
    plt.text(0, att.max() - i / 3, s=m, c=col, label=col)

plt.show()
# print(TYPE_MAPPING)

In [ ]:
# to_drop = [12, 20, 37, 58, 63, 65, 67, 70]
# idx = np.argwhere(x['type'][0][0].cpu().numpy() == 11).flatten()

idx = np.argwhere(x['type'][0][0].cpu().numpy() >= 0).flatten()

to_drop = np.argwhere(att[idx] <= 1).flatten()
print(to_drop)

plt.figure(figsize=(15, 5))
plt.axhline(1, c="g", alpha=0.5)
plt.bar(x=idx, height=att[idx], color=['r' if i in to_drop else "b" for i in idx])

[12 13 14 15 49 57 60]
[12 13 14 45 49 57 58 60]
[12 13 14 45 48 49 57 58 59 60]
[12 13 14 15 37 46 48 49 56 57 59 60 85 91 92 94 95]

In [ ]:
# y

## Training

In [ ]:
class Config:
    """
    Parameters used for training
    """
    # General
    seed = 42
    verbose = 1
    device = "cuda"
    save_weights = True

    # Data
    processed_folder = "processed_2/"
    max_len = 50

    # k-fold
    k = 4
    folds_file = f"../input/folds_{k}.csv"
    selected_folds = [0]  # , 1, 2, 3]

    # Model
    name = "transfo_rnn"
    pretrained_weights = None
    syncbn = False
    num_classes = 250
    
    embed_dim = 128
    transfo_dim = embed_dim * 3
    transfo_heads = 4
    drop_rate = 0.

    # Training    
    loss_config = {
        "name": "ce",
        "smoothing": 0.,
        "activation": "softmax",
        "aux_loss_weight": 0.,
        "activation_aux": "softmax",
    }

    data_config = {
        "batch_size": 64,
        "val_bs": 64,
        "use_len_sampler": True,
    }

    optimizer_config = {
        "name": "AdamW",
        "lr": 5e-3,
        "warmup_prop": 0.,
        "betas": (0.9, 0.999),
        "max_grad_norm": 10.,
    }

    epochs = 100

    use_fp16 = True

    verbose = 1
    verbose_eval = 250

    fullfit = False
    n_fullfit = 1

In [ ]:
DEBUG = True
log_folder = None
run = None

In [ ]:
if not DEBUG:
    log_folder = prepare_log_folder(LOG_PATH)
    print(f"Logging results to {log_folder}")
    config_df = save_config(Config, log_folder + "config.json")
    create_logger(directory=log_folder, name="logs.txt")
#     run = init_neptune(Config, log_folder)

df = prepare_data(DATA_PATH, Config.processed_folder)
init_distributed(Config)

preds = k_fold(Config, df, log_folder=log_folder, run=run)

Done ! 